# Elasticsearch reference architecture for Agentic applications

This notebook in a guide application for building agentic applications using Elasticsearch Agentic Builder as tool provider and LangChain with LangGraph as workflow engine. This notebook is based on the Elastic Labs Blog post [Elasticsearch reference architecture for Agentic applications](https://www.elastic.co/search-labs/blog/elasticsearch-reference-architecture-for-agentic-applications).

![Architecture diagram](./arch_diagram.png)

## Use Case: Security Vulnerability Agent

The Security Vulnerability Agent identifies potential risks based on a user's question by combining three complementary layers:

1. **Semantic search** with embeddings over an internal knowledge base of past incidents, configurations, and known vulnerabilities to retrieve relevant historical evidence.
2. **Internet search** for newly published advisories or threat intelligence that may not yet exist internally.
3. **LLM correlation** that correlates and prioritizes both internal and external findings, evaluates their relevance to the user's specific environment, and produces a clear explanation along with potential mitigation steps.

# Install dependencies and importing packages

In [2]:
%pip install langchain langchain-mcp-adapters langchain-openai langgraph elasticsearch dotenv rich -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import os

from typing import TypedDict

import requests
from dotenv import load_dotenv

from elasticsearch import Elasticsearch, helpers

from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from pydantic import BaseModel, Field

from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

console = Console()

## Environment Setup

Throughout the application we use common items such as the Elasticsearch client and environment variables. 


In [4]:
load_dotenv()

ELASTICSEARCH_ENDPOINT = os.getenv("ELASTICSEARCH_ENDPOINT")
ELASTICSEARCH_API_KEY = os.getenv("ELASTICSEARCH_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
KIBANA_URL = os.getenv("KIBANA_URL")

INDEX_NAME = "security-vulnerabilities"
KIBANA_HEADERS = {
    "kbn-xsrf": "true",
    "Content-Type": "application/json",
    "Authorization": f"ApiKey {ELASTICSEARCH_API_KEY}",
}  # Useful for Agent Builder setup API calls

MCP_ENDPOINT = f"{KIBANA_URL}/api/agent_builder/mcp"

## Elasticsearch Client

Initialize the Elasticsearch client to interact with your Elasticsearch cluster.


In [5]:
es_client = Elasticsearch(ELASTICSEARCH_ENDPOINT, api_key=ELASTICSEARCH_API_KEY)

## Agent Builder Tool Creation

Create a tool specialized in security queries that will perform semantic search. This tool will be available through the Agent Builder MCP server.

In [6]:
security_search_tool = {
    "id": "security-semantic-search",
    "type": "index_search",
    "description": "Search internal security documents including incident reports, pentests, internal CVEs, security guidelines, and architecture decisions. Uses semantic search powered by ELSER to find relevant security information even without exact keyword matches. Returns documents with severity assessment and affected systems.",
    "tags": ["security", "semantic", "vulnerabilities"],
    "configuration": {
        "pattern": INDEX_NAME,
    },
}

try:
    response = requests.post(
        f"{KIBANA_URL}/api/agent_builder/tools",
        headers=KIBANA_HEADERS,
        json=security_search_tool,
    )

    if response.status_code == 200:
        print("✅ Security semantic search tool created successfully")
    else:
        print(f"Response: {response.text}")
except Exception as e:
    print(f"❌ Error creating tool: {e}")

Response: {"statusCode":400,"error":"Bad Request","message":"Tool with id security-semantic-search already exists","attributes":{}}


## Index Mapping

To define the data structure, we need to create an index with appropriate mappings. We are creating a `semantic_text` field to perform semantic search using the information from the fields marked with the `copy_to` property. This enables the ELSER model to generate embeddings for semantic search.


In [7]:
index_mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text", "copy_to": "semantic_field"},
            "content": {"type": "text", "copy_to": "semantic_field"},
            "doc_type": {"type": "keyword", "copy_to": "semantic_field"},
            "severity": {"type": "keyword", "copy_to": "semantic_field"},
            "affected_systems": {"type": "keyword", "copy_to": "semantic_field"},
            "date": {"type": "date"},
            "semantic_field": {"type": "semantic_text"},
        }
    }
}

if es_client.indices.exists(index=INDEX_NAME) is False:
    es_client.indices.create(index=INDEX_NAME, body=index_mapping)
    print(f"✅ Index '{INDEX_NAME}' created with semantic_text field for ELSER")
else:
    print(f"ℹ️  Index '{INDEX_NAME}' already exists, skipping creation")

ℹ️  Index 'security-vulnerabilities' already exists, skipping creation


## Data Ingestion

With the mapping definition, we can ingest the data using the bulk API. 


In [94]:
def build_bulk_actions(documents, index_name):
    for doc in documents:
        yield {"_index": index_name, "_source": doc}


try:
    with open("dataset.json", "r") as f:
        security_documents = json.load(f)

    success, failed = helpers.bulk(
        es_client,
        build_bulk_actions(security_documents, INDEX_NAME),
        refresh=True,
    )
    print(f"📥 {success} documents indexed successfully")

except Exception as e:
    print(f"❌ Error during bulk indexing: {str(e)}")

📥 6 documents indexed successfully


## LangChain MCP Client

Here we create an MCP client using LangChain to consume the Agent Builder tools. The Agent Builder MCP server is available at `{KIBANA_URL}/api/agent_builder/mcp` and exposes Elasticsearch data and Agent Builder tools, acting strictly as a tools provider.


In [8]:
client = MultiServerMCPClient(
    {
        "agent-builder": {
            "transport": "streamable_http",
            "url": MCP_ENDPOINT,
            "headers": {"Authorization": f"ApiKey {ELASTICSEARCH_API_KEY}"},
        }
    }
)

tools = await client.get_tools()

print(f"📋 MCP Tools available: {[t.name for t in tools]}")

📋 MCP Tools available: ['platform_core_search', 'platform_core_get_document_by_id', 'platform_core_execute_esql', 'platform_core_generate_esql', 'platform_core_get_index_mapping', 'platform_core_list_indices', 'platform_core_index_explorer', 'security-semantic-search']


## Agent Creation

Create an agent that selects the appropriate tool based on the user input. The agent is configured with a system prompt that defines it as a cybersecurity expert specializing in infrastructure security.


In [ ]:
reasoning = {"effort": "low"}

llm = ChatOpenAI(
    model="gpt-5.2-2025-12-11", reasoning=reasoning, openai_api_key=OPENAI_API_KEY
)

system_prompt = """You are a cybersecurity expert specializing in infrastructure security.

        Your role is to:
        1. Analyze security queries from users
        2. Search internal security documents (incidents, pentests, CVEs, guidelines)
        3. Provide actionable security recommendations
        4. Assess vulnerability severity and impact

        When responding:
        - Always search internal documents first using the agent builder tools
        - Provide specific, technical, and actionable advice
        - Cite relevant internal incidents and documentation
        - Assess severity (critical, high, medium, low)
        - Recommend immediate mitigation steps

        Be concise but comprehensive. Focus on practical security guidance."""

agent = create_react_agent(llm, tools)

## Agent State Definition

We define the application state. This state will be passed through the LangGraph workflow nodes, allowing each node to read and update the state as needed.


In [ ]:
# Define agent state
class AgentState(TypedDict):
    query: str
    agent_builder_response: dict
    internet_results: list
    final_response: str
    needs_internet_search: bool

## Internet Search Tool

Create a tool that searches the internet for newly published advisories or threat intelligence that may not yet exist internally. This tool uses the Serper API to search external sources for CVE, advisories, and security intelligence.


In [20]:
@tool("internet_search_tool")
def internet_search_tool(query: str, top_k: int = 3) -> list:
    """
    Search external sources using Serper API for CVE, advisories, and security intelligence.
    """

    url = "https://google.serper.dev/search"
    payload = {"q": query + " CVE security vulnerability", "num": top_k}
    headers_serper = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}

    try:
        response = requests.post(url, json=payload, headers=headers_serper)
        response.raise_for_status()
        data = response.json()

        results = []
        for item in data.get("organic", [])[:top_k]:
            results.append(
                {
                    "title": item.get("title"),
                    "snippet": item.get("snippet"),
                    "link": item.get("link"),
                    "source": "Web Search",
                }
            )
        return results
    except Exception as e:
        print(f"⚠️  Error in Serper API: {e}")
        return []

## LangGraph Workflow Nodes

We use LangGraph to define a workflow capable of making decisions, running tool calls, and summarizing results. The workflow consists of four main nodes:

- **call_agent_builder_semantic_search**: Queries internal documentation using the Agent Builder MCP server and stores the retrieved messages in the state.
- **decide_internet_search**: Analyzes the internal results and determines whether an external search is required.
- **perform_internet_search**: Runs an external search using the Serper API when needed.
- **generate_final_response**: Correlates internal and external findings and produces a final, actionable cybersecurity analysis for the user.




In [ ]:
# Node 1: Call Agent Builder semantic search tool via MCP
async def call_agent_builder_semantic_search(state: AgentState) -> AgentState:
    query = state["query"]
    console.print(f"🔍 [cyan]Searching internal docs for:[/cyan] {query}")

    response = await agent.ainvoke(
        {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query},
            ]
        }
    )

    messages = response.get("messages", [])
    final_message = messages[-1] if messages else None

    if final_message:
        agent_response_text = final_message.content
        console.print(
            f"✅ [green]Agent response:[/green] {agent_response_text[:200]}..."
        )

        state["agent_builder_response"] = {
            "response": {"message": agent_response_text},
            "raw_messages": messages,
        }
    else:
        console.print("⚠️  [yellow]No response from agent[/yellow]")
        state["agent_builder_response"] = {}

    return state


# Node 2: Decide if internet search needed
async def decide_internet_search(state: AgentState) -> AgentState:
    """
    Decide if we need additional internet search based on Agent Builder results. If yes, the state will have a new key called "needs_internet_search" with a boolean value.
    """
    # Extract content from Agent Builder response
    ab_response = state["agent_builder_response"]
    agent_response = (
        ab_response.get("response", {}).get("message", "") if ab_response else ""
    )

    prompt = f"""
      User query: "{state['query']}"
      
      Elasticsearch MCP server found the following internal information:
      {agent_response[:500] if agent_response else 'No response'}...
      
      Should we search external sources (CVE databases, security advisories) for additional context?
      Consider:
      - Search external if query asks about specific CVE numbers or public vulnerabilities
      - Search external if internal findings mention vulnerabilities that might have public advisories
      - Don't search external if internal findings are comprehensive and specific to company systems
    """

    # Structured output for decision making
    class SearchDecision(BaseModel):
        """Decision on whether additional internet search is needed."""

        needs_internet_search: bool = Field(
            description="Whether we need to search external CVE databases and security advisories to complement internal findings"
        )
        reasoning: str = Field(description="Brief explanation of the decision")

    llm_with_structure = llm.with_structured_output(SearchDecision)

    decision: SearchDecision = llm_with_structure.invoke(prompt)
    state["needs_internet_search"] = decision.needs_internet_search

    status = (
        "✅ [green]Yes[/green]"
        if state["needs_internet_search"]
        else "❌ [red]No[/red]"
    )
    console.print(f"🤔 [bold]Internet search needed:[/bold] {status}")
    console.print(f"   [dim]{decision.reasoning}[/dim]")

    return state


# Node 3: Search internet for additional information
async def perform_internet_search(state: AgentState) -> AgentState:
    if not state["needs_internet_search"]:
        console.print("ℹ️  [blue]Internet search not needed, skipping.[/blue]")
        state["internet_results"] = []
        return state

    query = state["query"]
    console.print(f"🌐 [cyan]Performing internet search for:[/cyan] {query}")

    results = internet_search_tool.invoke(query)
    state["internet_results"] = results

    console.print(f"🔎 [green]Internet search returned {len(results)} results.[/green]")
    for i, res in enumerate(results):
        console.print(f"  [{i}] [link]{res['link']}[/link] - {res['title']}")

    return state


# Node 4: Generate final response
async def generate_final_response(state: AgentState) -> AgentState:
    """
    Correlate Agent Builder findings with internet search and generate comprehensive response.
    """
    console.print("🧠 [magenta]Generating final correlated response...[/magenta]")

    ab_response = state["agent_builder_response"]
    internal_context = (
        ab_response.get("response", {}).get("message", "No internal findings")
        if ab_response
        else "No internal findings"
    )

    # Format internet results with links
    internet_results_formatted = []
    if state["internet_results"]:
        for i, r in enumerate(state["internet_results"], 1):
            internet_results_formatted.append(
                f"[{i}] {r['title']}\n    Snippet: {r['snippet']}\n    Link: {r['link']}"
            )

    external_context = (
        "\n\n".join(internet_results_formatted)
        if internet_results_formatted
        else "No internet information searched."
    )

    prompt = f"""
    You are a cybersecurity expert. Provide a comprehensive security analysis.

    **User query:** {state['query']}

    **Internal company findings (from internal Elasticsearch knowledge base):**
    {internal_context}

    **External security intelligence (from web sources):**
    {external_context}

    **Your task:**
    1. Correlate internal and external information
    2. Assess vulnerability severity and impact on company systems
    3. Provide specific, actionable mitigation steps
    4. Highlight any gaps in coverage or additional concerns

    **IMPORTANT - Citation Format:**
    - When referencing information from internal Elasticsearch, use: [internal knowledge]
    - When referencing information from external web sources, cite the link number like: [1], [2], etc.
    - Example: "Express 4.17 has a prototype pollution vulnerability [internal knowledge] which is documented as CVE-2022-24999 [1]"

    Be concise, technical, and actionable.
    """

    response_content = llm.invoke(prompt).content

    text_parts = []
    for item in response_content:
        if isinstance(item, dict):
            # Extract text from 'text' field if it exists
            if "text" in item:
                text_parts.append(item["text"])
            # If it's a string directly, use it
            elif isinstance(item.get("content"), str):
                text_parts.append(item["content"])
        elif isinstance(item, str):
            text_parts.append(item)
    response = "".join(text_parts) if text_parts else str(response_content)

    # Append external links section at the end if there are external results
    if state["internet_results"]:
        response += "\n\n---\n## Internet References\n"
        for i, r in enumerate(state["internet_results"], 1):
            response += f"[{i}] {r['title']}\n    URL: {r['link']}\n\n"

    state["final_response"] = response

    return state

## Workflow Definition

With the workflow nodes defined, we can now build the LangGraph workflow. The workflow connects the nodes with edges and conditional routing logic. The workflow starts with an internal search, then decides whether external search is needed, and finally generates a comprehensive response that correlates both internal and external findings.

![Workflow diagram](./svia_workflow.png)


In [32]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node(
    "call_agent_builder_semantic_search", call_agent_builder_semantic_search
)
workflow.add_node("decide_internet_search", decide_internet_search)
workflow.add_node("perform_internet_search", perform_internet_search)
workflow.add_node("generate_response", generate_final_response)

workflow.add_edge(START, "call_agent_builder_semantic_search")
workflow.add_edge("call_agent_builder_semantic_search", "decide_internet_search")


def should_search_in_internet(state: AgentState) -> str:
    """Route to internet search or directly to generate response"""
    if state["needs_internet_search"]:
        return "perform_internet_search"

    return "generate_response"


workflow.add_conditional_edges(
    "decide_internet_search",
    should_search_in_internet,
    {
        "perform_internet_search": "perform_internet_search",
        "generate_response": "generate_response",
    },
)

workflow.add_edge("perform_internet_search", "generate_response")
workflow.add_edge("generate_response", END)

compiled_workflow = workflow.compile()

## Generating the workflow diagram image (optional)

In [33]:
try:
    # Generate the graph visualization
    png_data = compiled_workflow.get_graph().draw_mermaid_png()
    output_path = "svia_workflow.png"

    # Save to file
    with open(output_path, "wb") as f:
        f.write(png_data)

    print(f"📊 Workflow diagram saved to: {output_path}")
except Exception as e:
    print(f"⚠️  Could not generate workflow diagram: {e}")

📊 Workflow diagram saved to: svia_workflow.png


# Query execution

In this section we execute the workflow with a sample query.

In [ ]:
query = "We are using Node.js with Express 4.17 for our API gateway. Are there known prototype pollution or remote code execution vulnerabilities?"

console.print(
    Panel.fit(f"🔐 [bold cyan]QUERY:[/bold cyan] {query}", border_style="cyan")
)

initial_state = AgentState(
    {
        "query": query,
    }
)

result = await compiled_workflow.ainvoke(initial_state)

console.print("\n")
console.print(
    Panel(
        Markdown(result["final_response"]),
        title="📋 FINAL RESPONSE",
        border_style="green",
        padding=(1, 2),
    )
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔐 QUERY: We are using Node.js with Express 4.17 for our API gateway. Are there known prototype pollution or    │
│ remote code execution vulnerabilities?                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔍 Searching internal docs for: We are using Node.js with Express 4.17 for our API gateway. Are there known 
prototype pollution or remote code execution vulnerabilities?

✅ Agent response: [{'id': 'rs_016c81a7dd82f49d00694f1d15aa888190ad8c2bc9dbb3aa4e', 'summary': [], 'type': 
'reasoning'}, {'type': 'text', 'text': "Yes—**Express 4.17.x is in the risk zone for prototype-pollution–style 
issues**, and in our environment it has already been associated with **RCE** when combined with typical 
middleware/dependency patterns.\n\n## What we know internally (most relevant)\n- **Incident Report: “Node.js 
Express 4.17 Prototype Pollution RCE” (2024-03-15)** — *severity: critical*  \n  - Affected: `api-gateway-prod`, 
`api-gateway-staging`  \n  - Vector: **prototype manipulation via JSON payloads in POST requests**, impacting 
middleware that processes user input  \n  - Outcome: **Remote Code Execution (RCE)**  \n  - Mitigations recorded: 
**upgrade to Express 4.18.2+**, add input validation, and **freeze critical objects**  \n  - Internal note: 
“Related to **CVE-2022-24999**”  \n  Source: internal incident report in `security-vulnerabilities` (ID 
`lElHzJoBpQ2yiOd-PXCe`)\n\n## Practical interpretation (Express + ecosystem)\n- Express itself typically doesn’t 
“RCE” on its own, but **prototype pollution in request parsing/merging paths** (JSON bodies, querystring parsing, 
deep merge utilities, etc.) can become **RCE** when:\n  - polluted properties influence template engines, command 
construction, dynamic `require`, logging formatters, auth logic, or other downstream code paths\n  - middleware 
uses unsafe deep-merge patterns on untrusted input\n\n## Severity assessment (for an API gateway on 4.17.x)\n- 
**Severity: High → Critical**, depending on whether any request data is deep-merged/used in dynamic execution 
paths.\n- Given our March 2024 incident on the same stack, treat it as **Critical until proven otherwise**.\n\n## 
Immediate mitigation steps (actionable)\n1. **Upgrade Express**: move off 4.17.x to **4.18.2+** (or latest 4.18.x).
\n2. **Lock and update transitive deps**:\n   - Run `npm audit` (and/or Snyk) and ensure packages commonly involved
in pollution chains are updated (e.g., querystring parsing like `qs`, body parsing, and any deep-merge 
utilities).\n3. **Block prototype-pollution keys at the edge** (gateway layer):\n   - Reject payloads containing 
`__proto__`, `prototype`, `constructor` anywhere in JSON/query/body.\n   - Enforce strict JSON schema validation 
(e.g., AJV) for all externally reachable routes.\n4. **Harden Express parsing** (where feasible):\n   - Consider 
using the **simple query parser** to avoid complex parsing behaviors:  \n     `app.set('query parser', 
'simple')`\n5. **Runtime hardening (Node.js)**:\n   - Use Node’s proto hardening flag if compatible with your 
runtime: `--disable-proto=throw` (or `delete`) to reduce prototype abuse impact.\n6. **Confirm no unsafe 
merges**:\n   - Search for `lodash.merge`, `_.merge`, `deepmerge`, object spread merges of `req.body` into config 
objects, etc. Replace with safe merges that ignore prototype keys, or use `Object.create(null)` maps.\n\nIf you 
tell me **your Node.js version**, and whether you use **express.json/body-parser**, `qs` (extended query parsing), 
or any deep-merge utilities in the gateway, I can narrow this to a concrete “likely exploitable / not likely 
exploitable” assessment and a focused patch plan.", 'annotations': [], 'id': 
'msg_016c81a7dd82f49d00694f1d20e140819090489a1f8b73b3a9'}]...

🤔 Internet search needed: ✅ Yes

The user asked about known/public vulnerabilities in Express 4.17, and internal notes reference a specific CVE 
(CVE-2022-24999) and RCE claims that should be verified and contextualized against public advisories. External 
CVE/advisory lookup is needed to confirm affected versions, exploitability, and recommended fixed versions, and to 
distinguish Express core issues from ecosystem/transitive dependency issues (e.g., qs/body-parser).

...

[0] https://www.usenix.org/system/files/usenixsecurity23-shcherbakov.pdf - [PDF] Prototype Pollution Leads to 
Remote Code Execution in Node.js

[1] https://developer.mozilla.org/en-US/docs/Web/Security/Attacks/Prototype_pollution - JavaScript prototype 
pollution - Security - MDN Web Docs

[2] https://nvd.nist.gov/vuln/detail/CVE-2025-57353 - CVE-2025-57353 - NVD

🧠 Generating final correlated response...

╭─────────────────────────────────────────────── 📋 FINAL RESPONSE ───────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                1) Correlation of internal + external information                                │
│                                                                                                                 │
│   • Internal evidence of exploitability in your environment: We have an internal incident (“Node.js Express     │
│     4.17 Prototype Pollution RCE”, 2024‑03‑15) showing Express 4.17.x–based API gateways were compromised via   │
│     prototype manipulation in JSON POST payloads, resulting in RCE, with mitigations recorded as upgrading      │
│     Express to 4.18.2+, adding input validation, and freezing critical objects [internal knowledge].            │
│   • External validation of the technique and impact: Prototype pollution is a well-documented class of          │
│     vulnerability in JavaScript where an attacker can modify properties on an object’s prototype chain [2].     │
│     Research also documents that prototype pollution can be chained into Remote Code Execution in Node.js       │
│     depending on downstream gadgetry (e.g., dynamic evaluation, unsafe templating/configuration, or module      │
│     loading patterns) [1].                                                                                      │
│   • Key takeaway: Even if Express core is not “an RCE by itself,” Express 4.17.x in a typical gateway stack     │
│     (parsers + deep merges + middleware) is in a high-risk configuration for prototype-pollution chains, and    │
│     in our company it has already been observed to reach RCE [internal knowledge], consistent with external     │
│     research [1][2].                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
│                                    2) Severity and impact on company systems                                    │
│                                                                                                                 │
│  Severity: Critical (until disproven by code/path review).                                                      │
│  Rationale:                                                                                                     │
│                                                                                                                 │
│   • We have a confirmed internal RCE incident on the same major version line (4.17.x) and same role (API        │
│     gateway) [internal knowledge].                                                                              │
│   • Prototype pollution often starts as integrity compromise (e.g., auth bypass, logic manipulation) and can    │
│     become RCE if any “gadgets” exist in the code/dependency chain (template engines, dynamic require, unsafe   │
│     command building, logging format strings, etc.) [1][2]. Gateways frequently have exactly these patterns     │
│     (routing/auth/config merging, plugin systems, observability middleware).                                    │
│                                                                                                                 │
│  Likely impact areas for an API gateway:                                                                        │
│                                                                                                                 │
│   • Full service compromise: RCE enables credential the

# Cleanup


In [ ]:
# Delete the index
es_client.indices.delete(index=INDEX_NAME)

ObjectApiResponse({'acknowledged': True})